In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy import stats

In [2]:
data = pd.read_pickle('df_tesco_marker_V3.pkl')

In [3]:
data.shape

(61990, 56)

In [4]:
data.columns

Index(['Site Number', 'Site Name', 'Transaction Date', 'Total Sales Value',
       'Total Baskets', 'Sales Baskets With Fuel', 'Baskets With Fuel',
       'Site ID', 'PBL', 'CRUSO_priceADO', 'CRUSO_priceUL95', 'volumeADO',
       'volumeUL95', 'sales_valADO', 'sales_valUL95', 'totalSalesEsso',
       'pct_dev', 'diff_shop_esso', 'SiteCode', 'SiteName', 'Date',
       'UL 95_ Sales Transaction Volume', 'UL 95_ Approved Price',
       'UL 95_ Marker Price', 'UL 95_ Approved Price Vs Marker Price',
       'ADO_ Sales Transaction Volume', 'ADO_ Approved Price',
       'ADO_ Marker Price', 'ADO_ Approved Price Vs Marker Price',
       'avgUL95VolSite', 'avgADOVolSite', 'diffAdoVol', 'diffUL95Vol',
       'diffAdoVolpct', 'diffUL95Volpct', 'diffAdoPricePct',
       'diffUL95PricePct', 'avgShopSalesFromFuelSiteWise',
       'pctChgShopSalesFromFuel', 'shopSales', 'avgShopSalesSiteWise',
       'ShopSalesPctChgFromAvg', 'AdoApprPriceChgPct', 'UL95AppPriceChgPct',
       'avgFuelSalesEssoSiteWi

In [5]:
# calcualting exclusive shop sales and shop slaes from fuel for sites:
data['exclShopSales'] = data['Total Sales Value']-data['Sales Baskets With Fuel']
data['shopSalesFromFuel'] = data['Sales Baskets With Fuel']-data['totalSalesEsso']

In [6]:
# selecting only required data for elasticity calculation:
seldf = data.loc[:,['Site Number', 'exclShopSales', 'UL 95_ Approved Price','UL 95_ Marker Price']]

In [7]:
seldf.drop(list(seldf[seldf['exclShopSales'] == 0].index), axis=0, inplace=True)

In [8]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [9]:
def get_plot(df):
    slope, intercept, r_value, p_value, std_err = stats.linregress(df['Actual log(P0/Pbar)'], y=df['Actual log(S0)'])
    line = slope*X+intercept

    df['Fitted log(S0)'] = np.concatenate(line)

    fig = go.Figure(data=[
        go.Scatter(x= df['Actual log(P0/Pbar)'], y=df['Actual log(S0)'],mode='markers', name='Actual', marker_color = 'gray'),
        go.Line(x= df['Actual log(P0/Pbar)'], y=df['Fitted log(S0)'], name='Fit', marker_color = 'firebrick')    
    ])

    fig.update_layout(
        #title = selectedCountry+' - Road Fuel Demand',
        yaxis_title="log(S0)",
        xaxis_title="log(P0/Pbar)",
        plot_bgcolor='rgba(0,0,0,0)'
    )

    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    #fig.show()
    #fig.write_image("fig1.pdf")
    return fig

In [10]:
elasticity = []
logSbar    = []
corrCoef   = []
pVal       = []
rmse       = []
r2         = []
mape       = []


for i in range(len(seldf['Site Number'].unique())): #for all sites 
    df = seldf[seldf['Site Number'] == seldf['Site Number'].unique()[i]]
    df.drop('Site Number', axis=1, inplace=True)
    df.columns = ['shopSales', 'P0', 'Pbar']
    data_df = pd.DataFrame()
    y = np.log(df['shopSales'].dropna().values).reshape(-1,1)
    pdiv = df['P0']/df['Pbar']
    X = np.log(pdiv.dropna()).values.reshape(-1,1)
    data_df = pd.concat([data_df, pd.DataFrame({'Actual log(S0)': y.flatten(), 'Actual log(P0/Pbar)': X.flatten()})], axis=0)
    #get_plot(data_df)
    slope, intercept, r_value, p_value, std_err = stats.linregress(data_df['Actual log(P0/Pbar)'], y=data_df['Actual log(S0)'])
    elasticity.append(slope)
    logSbar.append(intercept)
    corrCoef.append(r_value)
    pVal.append(p_value)
    rmse.append(round(np.sqrt(metrics.mean_squared_error(y, slope*X+intercept)),4))
    r2.append(round(metrics.r2_score(y, slope*X+intercept),4))
    mape.append(round(mean_absolute_percentage_error(y,slope*X+intercept),4))   

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



In [11]:
# df = data_df
# slope, intercept, r_value, p_value, std_err = stats.linregress(df['Actual log(P0/Pbar)'], y=df['Actual log(S0)'])
# line = slope*X+intercept

# df['Fitted log(S0)'] = np.concatenate(line)

# fig = go.Figure(data=[
#     go.Scatter(x= df['Actual log(P0/Pbar)'], y=df['Actual log(S0)'],mode='markers', name='Actual', marker_color = 'gray'),
#     go.Line(x= df['Actual log(P0/Pbar)'], y=df['Fitted log(S0)'], name='Fit', marker_color = 'firebrick')    
# ])

# fig.update_layout(
#     #title = selectedCountry+' - Road Fuel Demand',
#     yaxis_title="log(S0)",
#     xaxis_title="log(P0/Pbar)",
#     plot_bgcolor='rgba(0,0,0,0)'
# )

# fig.update_xaxes(showgrid=False)
# fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
# fig.show()

In [12]:
# slope, intercept, r_value, p_value, std_err = stats.linregress(data_df['Actual log(P0/Pbar)'], y=data_df['Actual log(S0)'])
# elasticity.append(slope)
# logSbar.append(intercept)
# corrCoef.append(r_value)
# pVal.append(p_value)
# rmse.append(round(np.sqrt(metrics.mean_squared_error(y, slope*X+intercept)),4))
# r2.append(round(metrics.r2_score(y, slope*X+intercept),4))
# mape.append(round(mean_absolute_percentage_error(y,slope*X+intercept),4))   

In [13]:
result_df = pd.DataFrame({'elasticity': elasticity, 'constant': logSbar, 'p-value':pVal, 'r2':r2, 'mape': mape, 'rmse':rmse})

In [14]:
result_df

elasticity  constant   p-value      r2    mape    rmse
0      0.369143  9.455309  0.804781  0.0002  1.9584  0.2196
1      0.549874  9.415112  0.441494  0.0018  0.7999  0.0933
2      2.477734  8.629204  0.033030  0.0142  1.0622  0.1143
3      1.024903  9.350568  0.190957  0.0054  0.8523  0.1038
4     -2.680043  9.080978  0.005367  0.0253  1.1722  0.1437
..          ...       ...       ...     ...     ...     ...
191    2.226809  9.046231  0.009298  0.0209  1.0621  0.1259
192   -0.860804  9.367596  0.226188  0.0046  0.8611  0.1057
193    1.978797  8.729241  0.000137  0.0482  1.0509  0.1203
194    0.844115  8.984076  0.050410  0.0117  0.8060  0.1034
195   -0.069899  9.574679  0.931054  0.0000  2.1210  0.2247

[196 rows x 6 columns]

In [15]:
result_df[result_df['p-value'] < 0.05].shape

(64, 6)

In [16]:
import plotly.express as px
fig = px.histogram(result_df, x="elasticity", nbins=20)
fig.show()

In [17]:
result_df.to_csv("UL95_excShopSales_elasticity")

In [18]:
result_df.describe()

elasticity    constant       p-value          r2        mape  \
count  196.000000  196.000000  1.960000e+02  196.000000  196.000000   
mean     0.064615    9.039442  3.007649e-01    0.013524    0.990629   
std      1.362573    0.278456  3.027978e-01    0.021107    0.210805   
min     -3.918740    8.006241  1.248684e-12    0.000000    0.423300   
25%     -0.677985    8.871491  1.726975e-02    0.001175    0.859275   
50%      0.079803    9.049163  1.941609e-01    0.005400    0.962550   
75%      0.847248    9.248115  5.412149e-01    0.017350    1.089450   
max      4.688291    9.870990  9.753155e-01    0.139500    2.121000   

             rmse  
count  196.000000  
mean     0.116895  
std      0.026981  
min      0.054800  
25%      0.102075  
50%      0.113100  
75%      0.125800  
max      0.270200